In [4]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import numpy as np
import optuna
import pickle

from simulate.simulate import simulate

from inference.loss import LossTeaching
from inference.train_minibatch import train_minibatch

from plot.plot_hist_loss import plot_loss
from plot.plot_posterior import plot_posterior

In [2]:
%config InlineBackend.figure_format = "retina"
sns.set()

In [3]:
data, truth = simulate(use_torch=True)

In [25]:
def objective(trial):
    
    try:
        kwargs = dict(
            n_epochs=200,
            initial_lr=trial.suggest_float("lr", 1e-5, 1e-1, log=True),
            optimizer_name="RMSprop", # trial.suggest_categorical("optimizer", ["Adam", "RMSprop", "SGD"]),
            constant_lr=False,        # trial.suggest_categorical("constant_lr", [True, False]),
            flow_length=trial.suggest_int("flow_length", 1, 40),
            n_sample=trial.suggest_int("n_sample", 1, 100),
            batch_size=1000
        )

        z_flow, theta_flow, hist_loss = train_minibatch(data, **kwargs)

        loss = LossTeaching()(
            z_flow=z_flow,
            theta_flow=theta_flow,
            n_sample=100,
            n_u=len(np.unique(data['u'])), 
            n_w=len(np.unique(data['w'])),
            **data).item()
    except Exception as e:
        print(e)
        loss = np.nan
    return loss

In [26]:
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=300)

study.best_params

[I 2021-06-03 10:51:32,984] A new study created in memory with name: no-name-34cfca8c-6139-4bc0-a063-89234da39218


  0%|          | 0/200 [00:00<?, ?it/s]

[I 2021-06-03 10:55:02,043] Trial 0 finished with value: 21578.97308430766 and parameters: {'lr': 5.355163669287129e-05, 'flow_length': 13, 'n_sample': 85}. Best is trial 0 with value: 21578.97308430766.


  0%|          | 0/200 [00:00<?, ?it/s]

[I 2021-06-03 10:58:17,124] Trial 1 finished with value: 8812.84525465688 and parameters: {'lr': 0.004962931342907581, 'flow_length': 17, 'n_sample': 49}. Best is trial 1 with value: 8812.84525465688.


...

  0%|          | 0/200 [00:00<?, ?it/s]

[I 2021-06-04 10:27:57,546] Trial 298 finished with value: 8577.605196084081 and parameters: {'lr': 0.0019792863895785285, 'flow_length': 34, 'n_sample': 32}. Best is trial 199 with value: 8532.422296609187.


  0%|          | 0/200 [00:00<?, ?it/s]

[I 2021-06-04 10:32:50,641] Trial 299 finished with value: 8571.684795882267 and parameters: {'lr': 0.0013157122887209063, 'flow_length': 33, 'n_sample': 37}. Best is trial 199 with value: 8532.422296609187.


{'lr': 0.0012386149007894485, 'flow_length': 34, 'n_sample': 38}

In [32]:
with open("bkp/optuna_study.p", "wb") as f:
    pickle.dump(study, f)

In [5]:
with open("bkp/optuna_study.p", "rb") as f:
    study = pickle.load(f)

In [9]:
fig = optuna.visualization.plot_optimization_history(study)
fig

In [11]:
fig.update_yaxes(range=[8000, 10000])

In [12]:
optuna.visualization.plot_intermediate_values(study)

[W 2021-06-07 11:58:23,789] You need to set up the pruning feature to utilize `plot_intermediate_values()`


In [14]:
optuna.visualization.plot_parallel_coordinate(study)

In [15]:
optuna.visualization.plot_contour(study)

In [17]:
fig = optuna.visualization.plot_slice(study)
fig

In [18]:
fig.update_yaxes(range=[8000, 10000])
fig

In [19]:
optuna.visualization.plot_param_importances(study)

In [20]:
optuna.visualization.plot_edf(study)